In [ ]:
import pandas as pd
import numpy as np
import datetime

## スタッフDB編集

In [ ]:
dct = {
    'IDm/UID': "UID",
    '職員番号': "staff_number", 
    '漢字氏名': "kanji_name", 
    'カタカナ': "kana_name", 
    '所属': "dept", 
    '職種': "job", 
    '所属コード': "dept_code", 
    '区分コード': "class_code", 
    '鍵データ': "key_data",
    'カードデータ': "internal_card_data", 
    '発行日': "issue_date", 
    '修正日': "update_date",
}

file_path = 'data/nyutai_data_20250328.csv'
df = pd.read_csv(file_path, encoding='cp932', header=None, dtype=object)
df.columns = dct.values()

In [ ]:
df.dropna(subset=['UID'], how='all', axis=0, inplace=True)
df.set_index('UID', drop=True, inplace=True)

df.issue_date = pd.to_datetime(df.issue_date, format="%Y/%m/%d")
df.update_date = pd.to_datetime(df.update_date, format="%Y/%m/%d")

In [ ]:
l = []
for uid, name in df.kanji_name.to_dict().items():
    if type(name) == str:
        name = name.replace('　　', ' ')
        name = name.replace('　 ', ' ')
        name = name.replace(' 　', ' ')
        name = name.replace('　', ' ')
        name = name.split(" ")
    else:
        name = [name] # nanの対応

    length = len(name)
    if length >= 3: # 3つ以上の場合は、最初の1つを姓、残りを名に分ける
        name = [name[0], (" ").join(name[1:])]
    elif length == 1: # 1つの場合は、名を空欄にする
        name = name + [np.nan]
    l.append([uid, name[0], name[1]])

In [ ]:
df_kanji = pd.DataFrame(l, columns=['UID', 'kanji_last_name', 'kanji_first_name'])
df_kanji.set_index('UID', drop=True, inplace=True)

In [ ]:
l = []
for uid, name in df.kana_name.to_dict().items():
    if type(name) == str:
        name = name.replace('  ', ' ')
        name = name.split(" ")
    else:
        name = [name] # nanの対応

    length = len(name)
    if length >= 3: # 3つ以上の場合は、最初の1つを姓、残りを名に分ける
        name = [name[0], (" ").join(name[1:])]
    elif length == 1: # 1つの場合は、名を空欄にする
        name = name + [np.nan]
    l.append([uid, name[0], name[1]])

In [ ]:
df_kana = pd.DataFrame(l, columns=['UID', 'kana_last_name', 'kana_first_name'])
df_kana.set_index('UID', drop=True, inplace=True)

In [ ]:
df_merge = pd.merge(df, df_kanji, on='UID', how='left')
df_merge = pd.merge(df_merge, df_kana, on='UID', how='left')

In [ ]:
df_merge.drop(columns=['kanji_name', 'kana_name'], inplace=True)
df_merge = df_merge.reindex(columns=[
    'staff_number', 'kanji_last_name', 'kanji_first_name', 'kana_last_name', 'kana_first_name', 
    'dept', 'job', 'dept_code', 'class_code', 'key_data', 'internal_card_data', 'issue_date', 'update_date'
])
df_merge['dept'] = df_merge['dept'].map(lambda x: x.replace("_", "")[1:] if isinstance(x, str) and "_" in x else x)

In [ ]:
dct = {
    "マスターＢ": "マスターB", #半角英字に統一
    "救急部": "救急科", #「部」を「科」に統一
    "食堂・胃腸外科": "食道・胃腸外科", #「食堂」は誤記と思われる
    "麻酔･疼痛･緩和医療": "麻酔・疼痛・緩和医療科", #統一
    "耳鼻咽喉・頭頚部外科": "耳鼻咽喉科・頭頸部外科", #「頭頚部」は「頭頸部」が一般的
    "耳鼻咽喉･頭頚部外": "耳鼻咽喉科・頭頸部外科", #「外」が誤記と思われる
    "婦人科･周産期母性": "婦人科・周産期母性科", #統一
    "歯科･顎･口腔外科": "歯科・顎・口腔外科", #統一
    "糖尿病・代謝・内分泌": "糖尿病・代謝・内分泌内科", #省略されていたため補完
    "ｱﾚﾙｷﾞｰ・膠原病内科": "アレルギー・膠原病内科", #全角表記に統一
    "総合医療教育研修ｾﾝﾀｰ": "総合医療教育研修センター", #全角表記に統一
    "総合医療教育センター": "総合医療教育研修センター", #「研修」が抜けていた
    "未来開発センター": "未来開拓センター", #「未来開発センター」と「未来開拓センター」が混在していたため統一
    "糖尿病･代謝･内分泌内科": "糖尿病・代謝・内分泌内科", #統一
    "ﾘﾊﾋﾞﾘﾃｰｼｮﾝ部": "リハビリテーション部", #全角表記に統一
    "ﾏｽｽﾍﾟｸﾄﾛﾒﾄﾘｰ検査診断学寄付研究部門": "マススペクトロメトリー検査診断学寄附研究部門", #全角表記と「寄付→寄附」に統一
    "ﾏｽｽﾍﾟｸﾄﾛﾒﾄﾘｰ検査診断学寄附研究部門": "マススペクトロメトリー検査診断学寄附研究部門", #全角表記に統一
    "こどものこころ診療": "こどものこころ診療部", #「部」が抜けていた
    "婦人科、周産期母性科": "婦人科・周産期母性科", #「、」を「・」に統一
    "乳腺･甲状腺外科": "乳腺・甲状腺外科", #統一
    "内視鏡ｾﾝﾀｰ": "内視鏡センター", #全角表記に統一
    "耳鼻咽喉･頭頸部外科": "耳鼻咽喉科・頭頸部外科", #統一
    "臨床工学ｾﾝﾀｰ": "臨床工学センター", #全角表記に統一
    "ＭＥ機器管理センター": "ME機器管理センター", #半角英字に統一
    "乳腺甲状腺外科": "乳腺・甲状腺外科", #統一
    "未来医療系事務部\u3000医学部": "未来医療系事務部 医学部", #特殊スペースを通常スペースに修正
}
    # "ｱﾚﾙｷﾞｰ・膠原病内科，糖尿病・代謝・内分泌内科": "アレルギー・膠原病内科・糖尿病・代謝・内分泌内科", #「，」を「・」に統一
    # "食道・胃腸外科、乳腺・甲状腺外科": "食道・胃腸外科・乳腺・甲状腺外科", #「、」を「・」に統一
    # "肝胆膵外科、心臓血管外科": "肝胆膵外科・心臓血管外科", #「、」を「・」に統一
    # 呼吸器内科/臨床腫瘍部
    # 総合診療科/ﾘﾊﾋﾞﾘﾃｰｼｮﾝ部
    # 救急部・集中治療部
    # 救急科・集中治療部
df_merge['dept'] = df_merge['dept'].map(lambda x: dct[x] if x in dct.keys() else x)

In [ ]:
today = datetime.date.today().strftime("%Y%m%d")
file_path = f'data/nyutai_data_processed_{today}.csv'
df_merge.to_csv(file_path, encoding='utf-8-sig', index=True, header=True)